In [9]:
import json
import csv
import pandas as pd
import re

In [10]:
import urllib.request as req
url = "https://www.ettoday.net/news_search/doSearch.php?keywords=%E8%B2%AC%E4%BB%BB%E6%B6%88%E8%B2%BB%E8%88%87%E7%94%9F%E7%94%A2"
request = req.Request(url,headers = {"User-Agent":"/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36"})
with req.urlopen(request) as response:
    data =response.read().decode("utf-8")

In [11]:
import requests

from bs4 import BeautifulSoup 
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")
result = soup.find_all("h2",limit=3) 
print(result)

[<h2><a href="https://www.ettoday.net/news/20230602/2511997.htm" onclick="_gaq.push(['_trackEvent', '搜尋-結果列表','責任消費與生產','https://www.ettoday.net/news/20230602/2511997.htm']);" target="_blank">輔大「世界大學影響力」排名大躍進　SDGs17和台大並列全台第四名</a></h2>, <h2><a href="https://www.ettoday.net/news/20230518/2501634.htm" onclick="_gaq.push(['_trackEvent', '搜尋-結果列表','責任消費與生產','https://www.ettoday.net/news/20230518/2501634.htm']);" target="_blank">「不塑之客」愛地球！向聯合國指標邁進　花蓮近250店家參與</a></h2>, <h2><a href="https://www.ettoday.net/news/20230309/2455439.htm" onclick="_gaq.push(['_trackEvent', '搜尋-結果列表','責任消費與生產','https://www.ettoday.net/news/20230309/2455439.htm']);" target="_blank">綠色環保是趨勢！花蓮力推自備餐具享優惠　45家業者響應</a></h2>]


In [12]:
link_list = list()
titles = soup.find_all("h2")
for title in titles:
    anchor = title.select_one("a")
    if anchor is not None:
        link_list.append(anchor.get("href"))
        #print(anchor.get("href"))
link_list 

['https://www.ettoday.net/news/20230602/2511997.htm',
 'https://www.ettoday.net/news/20230518/2501634.htm',
 'https://www.ettoday.net/news/20230309/2455439.htm',
 'https://www.ettoday.net/news/20230209/2437424.htm',
 'https://www.ettoday.net/news/20221111/2377810.htm',
 'https://www.ettoday.net/news/20221005/2351896.htm',
 'https://www.ettoday.net/news/20220812/2315078.htm',
 'https://www.ettoday.net/news/20220804/2309146.htm',
 'https://www.ettoday.net/news/20220426/2237784.htm',
 'https://www.ettoday.net/news/20220223/2194745.htm']

In [13]:
head_list = list()
titles = soup.find_all("h2")
for title in titles:
    anchor = title.select_one("a")
    if anchor is not None:
        head_list.append(anchor.getText())
        #print(anchor.getText())
print(head_list)

['輔大「世界大學影響力」排名大躍進\u3000SDGs17和台大並列全台第四名', '「不塑之客」愛地球！向聯合國指標邁進\u3000花蓮近250店家參與', '綠色環保是趨勢！花蓮力推自備餐具享優惠\u300045家業者響應', '元智《食是求適》團隊\u3000預訪剩食獲30萬行動獎金', '櫃買中心致力推動ESG！永續債券市場規模破3500億\u3000今年發行量再創高', '消費記住4口訣！美麗包裝拆開後成垃圾\u3000減少層層包裹愛地球', '佳世達「台灣永續行動獎」雙金一銀一銅肯定\u3000實踐SDGs大有所成！', '超商首創「循環經濟資源回收機」！拿電池、寶特瓶爽換購物金', '廢棄物找有緣人！花蓮「免廢市集」免費換物續資源', '台南展現淨零排放決心\u3000永續發展目標自願檢視報告出爐']


In [14]:
cleaned_list = [text.replace('\u3000', ' ') for text in head_list]
cleaned_list

['輔大「世界大學影響力」排名大躍進 SDGs17和台大並列全台第四名',
 '「不塑之客」愛地球！向聯合國指標邁進 花蓮近250店家參與',
 '綠色環保是趨勢！花蓮力推自備餐具享優惠 45家業者響應',
 '元智《食是求適》團隊 預訪剩食獲30萬行動獎金',
 '櫃買中心致力推動ESG！永續債券市場規模破3500億 今年發行量再創高',
 '消費記住4口訣！美麗包裝拆開後成垃圾 減少層層包裹愛地球',
 '佳世達「台灣永續行動獎」雙金一銀一銅肯定 實踐SDGs大有所成！',
 '超商首創「循環經濟資源回收機」！拿電池、寶特瓶爽換購物金',
 '廢棄物找有緣人！花蓮「免廢市集」免費換物續資源',
 '台南展現淨零排放決心 永續發展目標自願檢視報告出爐']

In [15]:
df = pd.DataFrame({'head': head_list})
df = df.assign(link=link_list)
df

,head,link
0,輔大「世界大學影響力」排名大躍進 SDGs17和台大並列全台第四名,https://www.ettoday.net/news/20230602/2511997.htm
1,「不塑之客」愛地球！向聯合國指標邁進 花蓮近250店家參與,https://www.ettoday.net/news/20230518/2501634.htm
2,綠色環保是趨勢！花蓮力推自備餐具享優惠 45家業者響應,https://www.ettoday.net/news/20230309/2455439.htm
3,元智《食是求適》團隊 預訪剩食獲30萬行動獎金,https://www.ettoday.net/news/20230209/2437424.htm
4,櫃買中心致力推動ESG！永續債券市場規模破3500億 今年發行量再創高,https://www.ettoday.net/news/20221111/2377810.htm
5,消費記住4口訣！美麗包裝拆開後成垃圾 減少層層包裹愛地球,https://www.ettoday.net/news/20221005/2351896.htm
6,佳世達「台灣永續行動獎」雙金一銀一銅肯定 實踐SDGs大有所成！,https://www.ettoday.net/news/20220812/2315078.htm
7,超商首創「循環經濟資源回收機」！拿電池、寶特瓶爽換購物金,https://www.ettoday.net/news/20220804/2309146.htm
8,廢棄物找有緣人！花蓮「免廢市集」免費換物續資源,https://www.ettoday.net/news/20220426/2237784.htm
9,台南展現淨零排放決心 永續發展目標自願檢視報告出爐,https://www.ettoday.net/news/20220223/2194745.htm
